In [2]:
# import findspark
    # findspark.init() # Nếu không dùng image PySpark
    
from pyspark.sql import SparkSession

# Cấu hình MinIO (S3)
spark = SparkSession.builder \
    .appName("BronzeIngest") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "admin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# 1. Đọc dữ liệu thô (giả định file đã có trong /data)
raw_df = spark.read.csv("./data/txn_data.csv", header=True, inferSchema=True)

# 2. Ghi vào Bronze (Delta Table trên MinIO)
bronze_path = "s3a://datalake/bronze/transactions"
raw_df.write.format("delta").mode("overwrite").save(bronze_path)
print(f"Bronze table saved to: {bronze_path}")

# Đóng Spark Session
spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/24 06:22:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/24 06:22:58 WARN SparkSession: Cannot use io.delta.sql.DeltaSparkSessionExtension to configure session extensions.
java.lang.ClassNotFoundException: io.delta.sql.DeltaSparkSessionExtension
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:592)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:467)
	at org.apache.spark.util.SparkClassUtils.classForName(SparkClassUtils.scala:41)
	at org.apache.spark.util.SparkClassUtils.classForName$(SparkClassUtils.scala:36)
	at org.apache.spark.util.Utils$.classForName(Utils.

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/workspace/data/txn_data.csv.

25/11/24 10:19:04 ERROR Inbox: Ignoring error
java.util.concurrent.RejectedExecutionException: Task java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask@3847d731[Not completed, task = java.util.concurrent.Executors$RunnableAdapter@493f617b[Wrapped task = org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend$StandaloneDriverEndpoint$$anon$2@29380799]] rejected from java.util.concurrent.ScheduledThreadPoolExecutor@68f02e61[Terminated, pool size = 0, active threads = 0, queued tasks = 0, completed tasks = 0]
	at java.base/java.util.concurrent.ThreadPoolExecutor$AbortPolicy.rejectedExecution(ThreadPoolExecutor.java:2065)
	at java.base/java.util.concurrent.ThreadPoolExecutor.reject(ThreadPoolExecutor.java:833)
	at java.base/java.util.concurrent.ScheduledThreadPoolExecutor.delayedExecute(ScheduledThreadPoolExecutor.java:340)
	at java.base/java.util.concurrent.ScheduledThreadPoolExecutor.schedule(ScheduledThreadPoolExecutor.java:562)
	at org.apache.spark.scheduler.clus

In [ ]:
# Khởi tạo Spark Session với các config tương tự như trên
    # ...
    
    # 1. Đọc stream từ Kafka
    kafka_stream_df = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") # Internal network
        .option("subscribe", "transactions_raw") \
        .load()
        
    # 2. Extract giá trị (Value là binary)
    raw_stream_df = kafka_stream_df.selectExpr("CAST(value AS STRING) as raw_json")
    
    # 3. Định nghĩa đường dẫn Checkpoint và đầu ra
    bronze_path_stream = "s3a://datalake/bronze/realtime_transactions"
    checkpoint_path = "s3a://datalake/checkpoints/realtime_txn"
    
    # 4. Ghi stream vào Bronze Delta (Dùng append mode)
    query = raw_stream_df.writeStream \
        .format("delta") \
        .outputMode("append") \
        .option("checkpointLocation", checkpoint_path) \
        .start(bronze_path_stream)

    # Chạy và chờ (ví dụ: 60 giây để xem data đổ vào)
    # query.awaitTermination(timeout=60)
    # query.stop()